In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import dask.array as da
import pyproj

from pycontrails import Flight, MetDataset
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.core.met_var import (
    AirTemperature,
    RelativeHumidity,
    SpecificHumidity,
    EastwardWind,
    NorthwardWind,
    VerticalVelocity,
)

In [2]:
# meteorological parameters
met_params = {
    "air_temperature": 240.0,  # K
    "specific_humidity": 0.001,  # 1
    "relative_humidity": 0.5,  # 1
    "eastward_wind": 0.0,  # m/s
    "northward_wind": 0.0,  # m/s
    "wind_shear": 0.005,  # 1/s
    "vertical_velocity": 0.0,  # m/s
}

In [3]:
# flight trajectory parameters
fl_params = {
    "t0_fl": "2022-03-02 21:00:00",  # flight start time
    "rt_fl": pd.Timedelta(minutes=30),  # flight run time
    "ts_fl": pd.Timedelta(minutes=1),  # flight time step
    "ac_type": "A320",  # aircraft type
    "ac0_speed": 100.0,  # m/s
    "ac_heading": 45.0,  # deg
    "ac0_coords0": (-0.8, -0.8, 11500),  # lat, lon, alt [deg, deg, m]
    "sep_dist": (5000, 2000, 0),  # dx, dy, dz [m]
    "n_ac": 5,  # number of aircraft
}

In [4]:
# plume dispersion parameters
plume_params = {"dt_integration": pd.Timedelta(minutes=5), "max_age": pd.Timedelta(hours=3)}

In [5]:
# chemistry sim parameters
chem_params = {
    "t0_chem": pd.to_datetime("2022-03-01 21:00:00"),  # chemistry start time
    "rt_chem": pd.Timedelta(days=5),  # chemistry runtime
    "ts_chem": pd.Timedelta(minutes=1),  # chemistry time step
    "lat_bounds": (-1.0, 1.0),  # lat bounds [deg]
    "lon_bounds": (-1.0, 1.0),  # lon bounds [deg]
    "alt_bounds": (14500, 15000),  # alt bounds [m]
    "hres_chem": 0.1,  # horizontal resolution [deg]
    "vres_chem": 100,  # vertical resolution [m]
}

In [6]:
# create lists for lats, lons, alts, and times based on chem params
lats = np.arange(
    chem_params["lat_bounds"][0], chem_params["lat_bounds"][1], chem_params["hres_chem"]
)

lons = np.arange(
    chem_params["lon_bounds"][0], chem_params["lon_bounds"][1], chem_params["hres_chem"]
)

alts = np.arange(
    chem_params["alt_bounds"][0], chem_params["alt_bounds"][1], chem_params["vres_chem"]
)

times = pd.date_range(
    start=chem_params["t0_chem"],
    end=chem_params["t0_chem"] + chem_params["rt_chem"],
    freq=chem_params["ts_chem"],
)

In [7]:
# generate artifical met dataset (boxm currently only supports zero-wind scenarios)
data_vars = {
    param: (
        ["longitude", "latitude", "level", "time"],
        da.full(
            (len(lons), len(lats), len(alts), len(times)),
            value,
            chunks=(len(lons), len(lats), len(alts), 100),
        ),
    )
    for param, value in met_params.items()
}


met = xr.Dataset(
    data_vars, coords={"longitude": lons, "latitude": lats, "level": alts, "time": times}
)

met = MetDataset(met)

met

MetDataset with data:

<xarray.Dataset> Size: 807MB
Dimensions:            (longitude: 20, latitude: 20, level: 5, time: 7201)
Coordinates:
  * longitude          (longitude) float64 160B -1.0 -0.9 -0.8 ... 0.7 0.8 0.9
  * latitude           (latitude) float64 160B -1.0 -0.9 -0.8 ... 0.7 0.8 0.9
  * level              (level) float64 40B 1.45e+04 1.46e+04 ... 1.49e+04
  * time               (time) datetime64[ns] 58kB 2022-03-01T21:00:00 ... 202...
    air_pressure       (level) float64 40B 1.45e+06 1.46e+06 ... 1.49e+06
    altitude           (level) float64 40B -2.922e+04 -2.932e+04 ... -2.96e+04
Data variables:
    air_temperature    (longitude, latitude, level, time) float64 115MB dask.array<chunksize=(20, 20, 5, 100), meta=np.ndarray>
    specific_humidity  (longitude, latitude, level, time) float64 115MB dask.array<chunksize=(20, 20, 5, 100), meta=np.ndarray>
    relative_humidity  (longitude, latitude, level, time) float64 115MB dask.array<chunksize=(20, 20, 5, 100), meta=np.ndarray>
    eastward_wind      (longitude, latitude, level, time) float64 115MB dask.array<chunksize=(20, 20, 5, 100), meta=np.ndarray>
    northward_wind     (longitude, latitude, level, time) float64 115MB dask.array<chunksize=(20, 20, 5, 100), meta=np.ndarray>
    wind_shear         (longitude, latitude, level, time) float64 115MB dask.array<chunksize=(20, 20, 5, 100), meta=np.ndarray>
    vertical_velocity  (longitude, latitude, level, time) float64 115MB dask.array<chunksize=(20, 20, 5, 100), meta=np.ndarray>

In [8]:
# generate flight trajectories
fl = []

lon0, lat0, alt0 = fl_params["ac0_coords0"]
heading = fl_params["ac_heading"]
dist = fl_params["ac0_speed"] * fl_params["rt_fl"].total_seconds()

# calculate the final coordinates
geod = pyproj.Geod(ellps="WGS84")
lon1, lat1, _ = geod.fwd(lon0, lat0, heading, dist)

# create flight object for leader flight and resample points according to ts_fl
df = pd.DataFrame()
df["longitude"] = [lon0, lon1]
df["latitude"] = [lat0, lat1]
df["altitude"] = [alt0, alt0]
df["time"] = [pd.to_datetime(fl_params["t0_fl"]), 
              pd.to_datetime(fl_params["t0_fl"]) + fl_params["rt_fl"]]

ts_fl_min = int(fl_params["ts_fl"].total_seconds() / 60)

fl0 = Flight(df).resample_and_fill(freq=f"{ts_fl_min}min")

fl.append(fl0)



Flight [4 keys x 31 length, 1 attributes]
	Keys: longitude, latitude, altitude, time
	Attributes:
	time                [2022-03-02 21:00:00, 2022-03-02 21:30:00]
	longitude           [-0.8, 0.343]
	latitude            [-0.8, 0.351]
	altitude            [11500.0, 11500.0]
	crs                 EPSG:4326


In [9]:
# estimate fuel flow and fuel burn using ps_model

In [10]:
# simulate plume dispersion and generate plume dataset

In [11]:
# run boxm simulation and generate chemistry dataset
# chem = Boxm()